# TDI Capstone Project, Part 1: Linking inspections to violations 

Unfortunately, no ID is given to unequivocally link violations to the inspections in which they were discovered. Therefore, a violation is linked ot an inspection (a "failed" inspection) if the inspection and violation occur at the same pollution source, if the violation occurs within `insp_viol_thresh_days` days of the previous inspection at the source, and if no other violations were reported between the inspection and violation of concern. `insp_viol_thresh_days` can be given any value. I chose 365 days because it gave the best model results. The purpose of the code in this notebook is to draw these connections. 

In [6]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt

from external_variables import data_path
start_date = pd.Timestamp(year=2003,month=1,day=1)
end_date = pd.Timestamp(year=2018,month=12,day=31)
insp_viol_thresh_days = 365
insp_event_thresh = 14

In [15]:
def load_raw_inspections(file_path,start_date=pd.Timestamp(year=2013,month=1,day=1),
                     end_date=pd.Timestamp(year=2018,month=12,day=31)):
    """Loads raw inspection data (directly from downloaded file), returns cleaned/parsed dataframe."""
    
    # Load data, parse dates manually (didn't work in load function for some reason?)
    fces_pces = pd.read_csv(file_path,dtype=str,parse_dates=['ACTUAL_END_DATE'])
    fces_pces['ACTUAL_END_DATE'] = fces_pces['ACTUAL_END_DATE'].apply(
                            lambda x: datetime.datetime.strptime(x,'%m-%d-%Y'))
    
    # Restrict dates to specified time period
    fces_pces = fces_pces[(fces_pces['ACTUAL_END_DATE']>=start_date) & 
                          (fces_pces['ACTUAL_END_DATE']<=end_date)]
    
    # Drop inspections without a date, sort. 
    fces_pces = fces_pces.dropna(axis=0,subset=['ACTUAL_END_DATE'])
    fces_pces = fces_pces.sort_values(by=['PGM_SYS_ID','ACTUAL_END_DATE'],axis=0)
    
    return fces_pces


def time_since_prev(df,date_col='ACTUAL_END_DATE',group_col='PGM_SYS_ID'): 
    """Adds new column: time since the previous inspection"""
    if group_col is not None: 
        time_diff = df.groupby(group_col)[date_col].diff()/pd.Timedelta('1 days')
        time_diff[time_diff.isnull()]=np.NaN
        time_diff = time_diff.astype(float)
    else: 
        time_diff = df[date_col].diff()/pd.Timedelta('1 days')
        time_diff[time_diff.isnull()]=np.NaN
        time_diff = time_diff.astype(float)
    return time_diff


def merge_events(df,time_since_col='time_since_prev',days_thresh=14): 
    """Merge inspection events that occur within 14 days of each other"""
    return df[((df[time_since_col]>days_thresh)) | (df[time_since_col].isna())]


def load_violations(file_path,start_date=pd.Timestamp(year=2003,month=1,day=1),
                     end_date=pd.Timestamp(year=2018,month=12,day=31)):
    """Loads inspection data directly from downloaded files, returns parsed and cleaned dataframe"""
    
    # Read in data
    violations = pd.read_csv(file_path,dtype='str')
    
    viol_dates = violations['EARLIEST_FRV_DETERM_DATE'].copy()
    viol_dates[violations['EARLIEST_FRV_DETERM_DATE'].isna()] = violations['HPV_DAYZERO_DATE'][
                                                            violations['EARLIEST_FRV_DETERM_DATE'].isna()]
    violations['VIOL_DATE'] = viol_dates
    violations = violations.dropna(axis=0,subset=['VIOL_DATE'])
    violations['VIOL_DATE'] = violations['VIOL_DATE'].apply(lambda x: 
                                                            datetime.datetime.strptime(x,'%m-%d-%Y'))
    
    # Restrict dates
    violations = violations[(violations['VIOL_DATE']>=start_date) & 
                            (violations['VIOL_DATE']<=end_date)]
    
    # Sort by SOURCE ID and date. 
    violations = violations.sort_values(by=['PGM_SYS_ID','VIOL_DATE'],axis=0)
    
    return violations


def link_viol_insp(inspections,violations,past_thresh=0,future_thresh=90):
    
    # Dictionary of violations--for easy access later. 
    viol_dict = {}
    for source_id,source_data in violations[['PGM_SYS_ID','VIOL_DATE','ACTIVITY_ID'
                                            ]].groupby('PGM_SYS_ID')[['ACTIVITY_ID','VIOL_DATE']]: 
        viol_dict[source_id] = source_data
    
    # Loop through all sources--link violations at that source to inspections. 
    pgmsysid,inspdate,violtf,actid = [],[],[],[]
    insp_dates_by_source = inspections[['PGM_SYS_ID','ACTUAL_END_DATE']].groupby('PGM_SYS_ID')['ACTUAL_END_DATE']
    for source_id,insp_dates in insp_dates_by_source:

        # Initialize lists/arrays to store for each inspection: 
        insp_dates = np.asarray(insp_dates) # inspection dates
        viol_tf = np.zeros(np.shape(insp_dates)) # inspection pass/fail
        act_ids = [np.NaN]*len(insp_dates) # violation activity ids (when applicable)
        viol_data = viol_dict.get(source_id)

        # If there has been a violation at the source, mark corresponding inspections as failed and 
        # record violation activity ID for the violation associated with the inspection. 
        if viol_data is not None: 
            viol_dates = list(viol_data['VIOL_DATE'])
            viol_actids = list(viol_data['ACTIVITY_ID'])
            for viol_date,viol_actid in zip(viol_dates,viol_actids): 
                time_diffs = (viol_date-insp_dates)/datetime.timedelta(days=1)
                time_diffs[(time_diffs<past_thresh) | (time_diffs>future_thresh)] = np.NaN
                try: 
                    ind_viol = np.nanargmin(time_diffs)
                    if viol_tf[ind_viol]==0:
                        viol_tf[ind_viol] = 1
                        act_ids[ind_viol] = str(viol_actid)
                except ValueError:
                    pass

        # Accumulate inspection results in long vectors. 
        pgmsysid.extend([source_id]*len(insp_dates))
        inspdate.extend(list(insp_dates))
        violtf.extend(list(viol_tf))
        actid.extend(list(act_ids))

    # Create dataframe with inspection results. Add results to inspections dataframe with merge. 
    for_join = pd.DataFrame({
        'PGM_SYS_ID':pgmsysid,
        'ACTUAL_END_DATE':inspdate,
        'VIOL':violtf,
        'VIOL_ACTID':actid
    })
    for_join['DATE_JOIN'] = for_join['ACTUAL_END_DATE'].apply(str)
    for_join = for_join.drop('ACTUAL_END_DATE',axis=1)
    inspections['DATE_JOIN'] = inspections['ACTUAL_END_DATE'].apply(str)
    new_df = pd.merge(inspections, for_join,  how='left', on=['PGM_SYS_ID','DATE_JOIN']).drop('DATE_JOIN',
                                                                                              axis=1)  
    # Adjust datatypes of new columns
    new_df['VIOL'] = new_df['VIOL'].astype(dtype=int)
    new_df['VIOL_ACTID'] = new_df['VIOL_ACTID'].astype(dtype=str)
    
    return new_df


## Step 1: Load inspection data

**Source:** ICIS-AIR inspections database ([download summary](https://echo.epa.gov/tools/data-downloads/icis-air-download-summary)).

**Known problems:** Texas, Louisiana, Nebraska, North Dakota might only have data after Oct 17, 2014. All other states should have data that's complete, back to to ~2003. Source: [known problems](https://echo.epa.gov/resources/echo-data/known-data-problems#alerts). 

In [4]:
inspections_path = os.path.join(data_path,'ICIS-AIR_downloads','ICIS-AIR_FCES_PCES.csv')
inspections = load_raw_inspections(inspections_path,start_date=start_date,end_date=end_date)

#a = inspections.groupby(inspections['ACTUAL_END_DATE'].dt.year).count()
#fig = plt.figure()
#ax = fig.add_subplot(111)
#a[['PGM_SYS_ID']].plot(ax=ax)
#ax.set_xlabel('Year')
#ax.set_ylabel('Number of inspections, nationwide')
#ax.set_ylim([0,85000])
#fig.savefig('./figs/num_inspections_by_year.png')

## Step 2: Clean inspection data
**Biggest problem:** Often, several inspections are reported at a single source on a single day, or over the course of a few days. I assume these are duplicates or are strongly associated with each other (i.e. are part of the same "inspection event"). I therefore decided to delete any inspections for which another inspection occurred at the same source within the previous `insp_event_thresh` days. I chose to set `insp_event_thresh` to 14 days. 

Days between consecutive inspection events, before cleaning: 
![time_btwn_insp](figs/time_between_insp.png)

In [10]:
# Days since previous inspection
inspections['time_since_prev'] = time_since_prev(inspections,date_col='ACTUAL_END_DATE',
                                                 group_col='PGM_SYS_ID')
inspections = merge_events(inspections,time_since_col='time_since_prev',days_thresh=insp_event_thresh)
inspections.head()

#fig = plt.figure()
#ax = inspections['time_since_prev'].hist(bins=np.linspace(0,100,101))
#ax.set_xlabel('Days since previous inspection at source')
#ax.set_ylabel('Frequency')
#ax.set_title('Times between inspections at sources, 2003-2018')
#fig.savefig('./figs/time_between_insp.png')

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES,time_since_prev
370,010000000901110001,3601073581,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,2017-01-20 00:00:00,CAANSPS,NaN
1453768,010000000901110001,3601047975,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,2017-03-10 00:00:00,NaN,49.0
373,01000000E000000001,3601532874,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-05-30 00:00:00,CAASIP,NaN
375,01000000E000000003,3601532881,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-04-18 00:00:00,CAASIP,NaN
376,01000000E000000004,3601545556,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-05-01 00:00:00,CAASIP,NaN


## Step 3: Load violation data

**Source:** ICIS-AIR inspections database ([download summary](https://echo.epa.gov/tools/data-downloads/icis-air-download-summary)).

**Known problems:** Texas, Louisiana, Nebraska, North Dakota might only have data after Oct 17, 2014. All other states should have data that's complete, back to to ~2003. Source: [known problems](https://echo.epa.gov/resources/echo-data/known-data-problems#alerts). 

In [11]:
violations_path = os.path.join(data_path,'ICIS-AIR_downloads','ICIS-AIR_VIOLATION_HISTORY.csv')
violations = load_violations(violations_path,start_date=start_date,end_date=end_date)

#a = violations.groupby(violations['VIOL_DATE'].dt.year).count()
#fig = plt.figure()
#ax = fig.add_subplot(111)
#a[['PGM_SYS_ID']].plot(ax=ax)
#ax.set_xlabel('Year')
#ax.set_ylabel('Number of violations, nationwide')
#ax.set_ylim([0,4000])
#fig.savefig('./figs/num_violations_by_year.png')

## Step 4: Clean violation data
Also "merge" violation events that were entered within `insp_event_thresh` days of each other--for the same reason as the inspection events. 

In [13]:
# Days since previous violation
violations['time_since_prev'] = time_since_prev(violations,date_col='VIOL_DATE',group_col='PGM_SYS_ID')
violations = merge_events(violations,time_since_col='time_since_prev',days_thresh=insp_event_thresh)

## Step 5: Create the links! 
Mark all "failed" inspections as failed. Include the violation Activity ID with the failed inspection. 

In [16]:
new_insp = link_viol_insp(inspections,violations,past_thresh=0,future_thresh=insp_viol_thresh_days)
new_insp.head()

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES,time_since_prev,VIOL,VIOL_ACTID
0,010000000901110001,3601073581,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,2017-01-20 00:00:00,CAANSPS,NaN,0,nan
1,010000000901110001,3601047975,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,2017-03-10 00:00:00,NaN,49.0,0,nan
2,01000000E000000001,3601532874,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-05-30 00:00:00,CAASIP,NaN,0,nan
3,01000000E000000003,3601532881,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-04-18 00:00:00,CAASIP,NaN,0,nan
4,01000000E000000004,3601545556,E,INS,Inspection/Evaluation,PCE,PCE On-Site,2018-05-01 00:00:00,CAASIP,NaN,0,nan


## Step 6: save the linked inspections/violations to file
Linked inspections will have name "processed_inspections_futurethresh_XX.csv" where XX is the value of `insp_viol_thresh_days`. 

In [11]:
insp_fname = 'processed_inspections_futurethresh_'+str(insp_viol_thresh_days)+'.csv'
new_insp.to_csv(os.path.join(data_path,insp_fname))
#viol_fname = 'processed_violations.csv'
#violations.to_csv(os.path.join(data_path,viol_fname))

In [ ]:
def TEST_linking_percentage(violations,new_insp):
    total_violations = violations['PGM_SYS_ID'].count()
    linked_violations = violations['VIOL'].sum()
    percent_linked = linked_violations/total_violations*100
    message = "Less than 60% of violations were linked to inspections. Percentage linked = "+\
                                                                        "%.1f" % percent_linked
    assert percent_linked>60, message
    
TEST_linking_percentage(violations,new_insp)

## Step 7: Verify linking strategy

In [134]:
import random
pgmid = random.choice(list(viol_dict.keys()))
viols = viol_dict.get(pgmid)
insp = new_insp[['PGM_SYS_ID','ACTUAL_END_DATE','VIOL','VIOL_ACTID']][new_insp['PGM_SYS_ID']==pgmid]
print(viols)
insp

               PGM_SYS_ID            VIOL_DATE ACTIVITY_ID
64403  CO0000000803700069  2006-01-19 00:00:00  3400388117
46827  CO0000000803700069  2015-08-17 00:00:00  3600482486


,PGM_SYS_ID,ACTUAL_END_DATE,VIOL,VIOL_ACTID
91931,CO0000000803700069,2007-06-18 00:00:00,0,nan
91932,CO0000000803700069,2011-07-21 00:00:00,0,nan
91933,CO0000000803700069,2015-05-19 00:00:00,1,3600482486
91934,CO0000000803700069,2017-11-13 00:00:00,0,nan


## Some additional relevant links

Relevant links: 
* [EPA ECHO](https://echo.epa.gov/)
* [EPA ECHO data downloads](https://echo.epa.gov/tools/data-downloads#downloads)
* [EPA--How we monitor compliance](https://www.epa.gov/compliance/how-we-monitor-compliance)
* [EPA--Air enforcement information](https://www.epa.gov/enforcement/air-enforcement#stationary)
* [EPA--Coal fired power plant data](https://www.epa.gov/airmarkets/coal-fired-power-plant-data)

Additional data sources: 
* [National emissions inventory](https://www.epa.gov/air-emissions-inventories/2014-national-emissions-inventory-nei-data) (Use 'data/frs_downloads/FRS_PROGRAM_LINKS.csv' to link my dataset with the NEI dataset). Facilities that emit more of a regulated pollutant might be more likely to violate standards? Some normalization factor might be helpful. 
* [EPA Air Quality Observations](https://aqs.epa.gov/aqsweb/airdata/download_files.html). Might be some relationship between observations of regulated pollutants and probability of violating standards? 
